In [1]:
from dotenv import load_dotenv
import os
import time
from langchain_community.utilities import SQLDatabase

load_dotenv()

True

## 1. vectorstore 구현


In [2]:
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document
import datetime

In [3]:
embedding = OpenAIEmbeddings()
vector_store = Chroma(persist_directory="../data/chroma_db", embedding_function=embedding)

In [4]:
## 더미데이터 저장

dummy_conversations = [
    {
        "session_id": "1",
        "role": "user",
        "timestamp": datetime.datetime.now().isoformat(),
        "content": "안녕하세요! 아메리카노 주문할 수 있나요?"
    },
    {
        "session_id": "1",
        "role": "assistant",
        "timestamp": datetime.datetime.now().isoformat(),
        "content": "네, 가능합니다! ICE로 하시겠어요? HOT으로 하시겠어요?"
    },
    {
        "session_id": "1",
        "role": "user",
        "timestamp": datetime.datetime.now().isoformat(),
        "content": "아이스로 한 잔 주세요!"
    },
    {
        "session_id": "1",
        "role": "assistant",
        "timestamp": datetime.datetime.now().isoformat(),
        "content": "주문이 완료되었습니다! 결제 방법을 선택해주세요."
    },
    {
        "session_id": "2",
        "role": "user",
        "timestamp": datetime.datetime.now().isoformat(),
        "content": "카페라떼 가격이 얼마인가요?"
    },
    {
        "session_id": "2",
        "role": "assistant",
        "timestamp": datetime.datetime.now().isoformat(),
        "content": "카페라떼는 2,900원입니다."
    }
]

def store_data(data_list):
    """더미 데이터를 ChromaDB에 저장"""
    documents = [
        Document(
            page_content=item["content"] if "content" in item else f"주문 내역: {item['items']}",
            metadata=item
        )
        for item in data_list
    ]
    vector_store.add_documents(documents)

# 데이터 저장 실행
store_data(dummy_conversations)



In [4]:
def store_conversation(session_id, user_message, ai_response):
    """유저 ID별 세션을 구분하여 대화 내역 저장"""

    timestamp = datetime.datetime.now().isoformat()

    # 유저 메시지 저장 (session_id 포함)
    user_doc = Document(
        page_content=user_message,
        metadata={"session_id": session_id, "role": "user", "timestamp": timestamp}
    )

    # AI 응답 저장 (session_id 포함)
    ai_doc = Document(
        page_content=ai_response,
        metadata={"session_id": session_id, "role": "assistant", "timestamp": timestamp}
    )

    # 벡터 저장소에 추가
    vector_store.add_documents([user_doc, ai_doc])

In [5]:
vector_store.get()

{'ids': ['c6dae5eb-3653-4264-8770-06334efa0462',
  'df4bed3a-173b-41da-8ad6-a2282e8d8b90',
  'dd2d2665-92fc-4c0b-b8c3-e20780619315',
  '072c9e2a-88b4-4905-9407-d7b775d968af',
  '31f4edac-2734-4ccc-afcb-d5b0988ad01a',
  'fde53191-f010-4ffc-97c3-11ada0f32b0e',
  '21a9696e-dbae-43f2-af90-b3d7591eac7c',
  '428ea681-f4cf-4380-a37b-ad7026de9398',
  '03f813c2-4562-4b50-8429-65cb30def3fb',
  '1f89918b-14cc-4b0f-8a19-c4a9d0139973',
  'f0c41d32-54f2-4ccd-b605-310b67826d6a',
  'e800a6a4-d175-4c43-a7ed-cff4e8257341',
  'c78fa6e8-6c53-48da-820b-7c5c92a2bc41',
  '1b8edaf0-da2b-4d03-8267-f59ec01dcf4b',
  '153e1bdc-9594-48f2-a8a1-c4e8f022a158',
  'c1a52b9d-e240-4991-afc6-5f8d305bdd39',
  '5a87f7e1-7b4f-4554-89f9-792fbe833cd5',
  'c3ab1515-3eaa-4173-87bc-05d95f6f300a',
  '2f30bfb4-9e38-4fc6-a21d-bfd3e8656625',
  '88fb8e77-9ac9-4e10-8f17-16fd94ef7a2c',
  '3befdc91-542a-40f5-9a93-51f3d73b94e5',
  '8d5554eb-2739-4ea3-9de3-40c1a7c6c628',
  'b036e0e2-71c1-4669-ace3-8c09e30ee38e',
  'e26aafb5-f0b2-4219-b16c-

## 2. Retriever 설정

In [6]:
session_id = "2"

vector_retriever = vector_store.as_retriever(search_type="mmr",search_kwargs={"k": 3, 'lambda_mult':0.1, "filter": {"session_id": session_id}})

## 3. prompt 구성

In [7]:
from langchain.prompts import FewShotPromptTemplate, PromptTemplate, ChatPromptTemplate


examples = [
    {"input": "아이스 아메리카노 한 잔 주세요.", "answer": "ICE 아메리카노 한 잔 맞으시죠? 드시고 가시나요? 가지고 가시나요?"},
    {"input": "카페라떼 한 잔 주세요.", "answer": "ICE로 하실까요? HOT으로 하실까요?"},
    {"input": "너 몇 살이니? ", "answer": "주문과 상관없는 질문은 대답해드릴 수 없습니다. 다른 질문을 해주시겠어요?"},
    {"input": "가지고 가겠습니다.", "answer": "감사합니다. 결제 수단을 선택해주세요."}
]

example_prompt = PromptTemplate(
    input_variables= ["input","answer"],
    template="입력: {input} \n 출력: {answer} \n"
    )

Few_shot_prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt = example_prompt,
    prefix ="""You are a helper agent who is built into the kiosk and tries to take orders.  
    Please be courteous and helpful to customers' questions and orders.
    Use the following pieces of retrieved tools to answer the question.
    If you don't know the answer, just say that you don't know.
    However, please only answer questions related to cafe work, such as inquiries about orders and menus.
    Answer in Korean.
    """, #프롬프트의 시작 부분
    suffix="고객: {input} \n AI: {agent_scratchpad}", #사용자 입력이 들어갈 부분
    input_variables=["input",'agent_scratchpad']
)

In [405]:
# prompt = PromptTemplate.from_template(
#     """
#     You are a helper agent who is built into the kiosk and tries to take orders.  
#     Please be courteous and helpful to customers' questions and orders.
#     Use the tools to answer the question.
#     If you don't know the answer, just say that you don't know.
#     However, please only answer questions related to cafe work, such as inquiries about orders and menus.
#     Answer in Korean.
    
#     고객 : {input}
    
#     tools : {agent_scratchpad}

#     답변 :
#     """

# )

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
            You are a helper agent who is built into the kiosk and tries to take orders. Please answer in 2 sentences.  
            Please be courteous and helpful to customers' questions and orders.  
            You should check out their beverage options like hot and iced.  

            The ordering process is as follows:  
            1. Select menu (Use `menu_db_sql_tool` to fetch menu details)  
            2. Confirm takeout option  
            3. Confirm additional menus or inquiries  
            4. Final confirmation of order with price information (MUST use `confirm_order` tool)  
            5. Select payment method  
            6. Wait for payment  
            7. After payment, send guidance message and 'Thank_you'.  

            ### TOOL RULES:
            - ALWAYS use `menu_db_sql_tool` to retrieve menu details, including price, instead of relying on assumptions.  
            - When the user asks about order history, use `chat_retriever_tool`.  
            - When the user confirms an order (e.g., "I want to buy this", "Proceed to checkout"), you MUST call `confirm_order` with the exact confirmed items.  
            - NEVER add items to the cart unless explicitly confirmed by the user.  

            ### GENERATION RULES:
            - When the user requests a product, use `menu_db_sql_tool` to fetch the exact product name stored in the database.  
            - ALWAYS use the official product name from the database when generating responses, even if the user's input is slightly different.  
            - If the user's input is similar but does not exactly match a product, assume they meant the closest match and use the name from the database.  
            - When confirming an order, list the products using their official names from the database.  
            (Example: "아이스 아메리카노 두 잔, 따뜻한 카페라떼 한 잔" → `ICE 아메리카노, ICE 아메리카노, HOT 카페라떼`)  

            ### IMPORTANT RULES:
            - DO NOT generate responses about menu items, prices, or cart contents without FIRST retrieving the required data from the tools.  
            - NEVER guess; if the answer is unknown, state that you don’t know.  
            - If the user asks for unavailable items, check the database first before responding.  
            - Answer in Korean.  
            - Your responses must be strictly based on the database. Any incorrect or fabricated information will result in a penalty.  

            """
        
        ),
        ("placeholder", "{page_content}"),
        ("placeholder", "{chat_history}"),
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}")

    ]
)

            # "you must answer in conversational form. Without any decoration such as symbols or dictionary forms."
            # """If the user specifies a quantity in their request (e.g., "3 cups of iced americano"), 
            # always separate the menu item and the quantity.
            # - Example: "생딸기 주스 3잔" → Menu: "생딸기 주스", Quantity: 3
            # - Example: "에스프레소 2잔" → Menu: "에스프레소", Quantity: 2"""
            # "If no quantity is specified, assume the default quantity is 1. \n"

prompt

ChatPromptTemplate(input_variables=['input'], optional_variables=['agent_scratchpad', 'chat_history', 'page_content'], input_types={'page_content': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages.chat.ChatMessageChunk, Tag(tag='ChatMessageChunk')], typing.Annotated[langchain_core.messages.system.SystemMessage

In [406]:
            # "You are a helper agent who is built into the kiosk and tries to take orders. Please answer in 2 sentences\n"
            # "Please be courteous and helpful to customers' questions and orders. \n"
            # "You should check out their beverage options like hot and iced. \n"
        
            # "The ordering process is as follows: Select menu - Confirm takeout - Confirm additional menus or inquiries -Final confirmation of order with price information(with run 'confirm_order' tool) -Select payment method - Wait for payment - After payment, guidance message and 'Thank_you' is sent. \n"
            
            # "TOOL RULES:"
            # "Use chat_retriever_tool for questions related to order history."
            # "Answer all questions about orders and menus using 'menu_db_sql_tool'."
            # "In particular, please use 'menu_db_sql_tool' to answer about price information.\n"

            # "GENARATION RULES:"
            # "When the user requests a product, use `menu_db_sql_tool` to fetch the exact product name stored in the database."
            # "ALWAYS use the product name as it appears in the database when generating responses, even if the user's input differs slightly."
            # "If the user's input does not match exactly but is similar to a product in the database, assume they meant the closest match and use the name in database. \n"
            # "When confirming an order, list the products using their official names from the database. (example, 아이스 아메리카노 두 잔, 따뜻한 카페라뗴 한 잔 -> [ICE 아메리카노 , ICE 아메리카노, HOT 카페라떼])"
            # "If the user confirms their order (e.g., 'I want to buy these items', 'Proceed to checkout'), call `confirm_order` with the confirmed items as the parameter."
            # "DO NOT add items to the cart unless explicitly confirmed by the user."

            # "IMPORTANT RULES:"
            # "Please only answer questions related to cafe work, such as inquiries about orders and menus."
            # "Do not guess; retrieve data using the tools before responding."
            # "Don't say you don't have it, just use the tools we provide."
            # "Nevertheless you don't know the answer, just say that you don't know."
            # "Answer in Korean.\n"
            # "Your responses must be strictly based on the database. Any incorrect or fabricated information will result in a penalty."

In [407]:
# from langchain_core.messages import SystemMessage, HumanMessage
# from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

# system_message = """
#             "system",
#             "You are a helper agent who is built into the kiosk and tries to take orders.\n"
#             "Please be courteous and helpful to customers' questions and orders. \n"
#             "You should check out their beverage options like hot and iced. \n"
#             "However, please only answer questions related to cafe work, such as inquiries about orders and menus."

#             "The ordering process is as follows: Select menu - Confirm takeout - Confirm additional menus or inquiries -Final confirmation of order with price information -Select payment method - Wait for payment - After payment, guidance message and 'Thank_you' is sent."

#             "Use chat_retriever_tool for questions related to order history. "
#             "Answer all questions about orders and menus using menu_db_sql_tool."
#             "In particular, please use menu_db_sql_tool to answer about price information."
#             "you must answer in conversational form. Without any decoration such as symbols or dictionary forms."
#             "Do not guess; retrieve data using the tools before responding."
#             "Don't say you don't have it, just use the tools we provide."
#             "Nnevertheless ,you don't know the answer, just say that you don't know."
#             "Answer in Korean.\n"
#             "Your responses must be strictly based on the database. Any incorrect or fabricated information will result in a penalty."

# """

# prompt = ChatPromptTemplate.from_messages([
#     SystemMessage(content=system_message),
#     MessagesPlaceholder(variable_name="chat_history"),
#     HumanMessage(content="{input}"),
#     MessagesPlaceholder(variable_name="agent_scratchpad")
# ])

# 4. llm 설정

In [408]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(temperature=0, model="gpt-4o-mini") 

chain = prompt | llm

# chain.invoke("몇 살이야?")

In [409]:
## 객체 설정
import sys
sys.path.append('C:/LLM_Project/Korean_AI_Kiosk_Agent/services')
from cart import Cart



now_cart = Cart()
query_buffer = []

In [410]:
now_cart.add_item('banana',4000,"screenntshot")

now_cart.clear_cart()
now_cart.items

[]

In [411]:
query_buffer

[]

# 5. Agnet 설정 및 tool 설정

In [412]:
from langchain.agents import initialize_agent, AgentType,OpenAIFunctionsAgent
from langchain.agents import create_tool_calling_agent
from langchain.tools.retriever import create_retriever_tool
from langchain.tools import Tool
from langchain.agents import AgentExecutor, create_sql_agent
from langchain_experimental.sql import SQLDatabaseChain
from langchain.sql_database import SQLDatabase
import sqlite3
from langchain_core.output_parsers import PydanticOutputParser
from langchain_openai import ChatOpenAI
import re
## tools custom

from langchain.tools.retriever import create_retriever_tool

chat_retriever_tool = create_retriever_tool(
    vector_retriever,    
    name='chat_search',
    description = "A tool to reference customer's previous conversations including their last ordered menu items and order history. Use this when you need to recall past interactions or order details."
)

# SQLite 데이터베이스 연결
db = SQLDatabase.from_uri("sqlite:///../data/Comfile_Coffee_DB.db")

sql_llm =ChatOpenAI(model='gpt-4o-mini',temperature=0.2)

# SQLDatabaseChain 생성



def make_query_and_run(text: str):
    db = SQLDatabase.from_uri("sqlite:///../data/Comfile_Coffee_DB.db")
    info = db.get_table_info()

    custom_prompt = PromptTemplate(
    input_variables=["query"],
    partial_variables={'info':info},
    template="""
    You are an expert in SQL and databases. Generate an SQL query based on the given question. 

    - The question may not always provide the exact menu name, so extract only the key characteristics.
    - If multiple keywords are provided, process them using `AND`.
    - Use `LIKE` and `%` to handle partial matches.
    - If the question includes "price" or "가격", do not return only the price—return all related information by using `SELECT *`.
    - Convert "따뜻한" or "뜨거운" to "HOT" in SQL queries.
    - Convert "차가운" or "아이스" to "ICE" in SQL queries.

    Return only the raw SQL query without any markdown formatting, code blocks, or explanations.

    You must reference the following database structure:
    {info}


        
    question: {query}
    """
    )

    sql_chain = custom_prompt | sql_llm
    result_query=sql_chain.invoke(text).content
    result = re.sub(r"```sql\n(.*?)\n```", r"\1", result_query, flags=re.DOTALL).strip()
    table =db.run(result)
    #query_buffer.append(table)
    return table
    

menu_db_sql_tool = Tool(
    name="menu_db_sql_tool",
    func=make_query_and_run,
    description="This tool retrieves menu information from the database based on user inquiries during the ordering process. It provides details such as item name, description, price, availability, and menu_id"
)


tools= [menu_db_sql_tool, chat_retriever_tool]

agent = create_tool_calling_agent(llm=llm,tools=tools,prompt=prompt)


In [413]:
# cart tool

# def store_query_result(items):
#     """
#     SQL 쿼리 결과를 임시 저장소(query_buffer)에 저장합니다.
#     """
#     query_buffer.append(items)  # 기존 데이터를 덮어씀
#     return query_buffer

# store_query_result_tool = Tool(
#     name="store_query_result",
#     description="Use this tool to temporarily store the results of menu_db_sql_tool. DO NOT add items to the cart immediately. This tool is only for temporary storage.",
#     func=store_query_result
# )
conn = sqlite3.connect("../data/Comfile_Coffee_DB.db")
cursor = conn.cursor()
menu_table=cursor.execute("SELECT * FROM menus").fetchall()
cursor.close()


def add_Cart(final_order_list):
    """
    주문이 확정되었는지 판단하고, 확정되었다면 장바구니(cart)에 저장합니다.
    """
    order_list= final_order_list.split(",")
    global now_cart # , query_buffer
    # for menu_table in query_buffer:
    #     for i in range(len(menu_table.split(","))//6):
    #         menu_price =  float(menu_table.split(",")[i*6+4])
    #         menu_name = re.sub(r"""['"]""","",menu_table.split(",")[i*6+1]).strip()
    #         image_url = re.sub(r"[^a-zA-Z0-9가-힣/._ ]","",menu_table.split(",")[i*6+5]).strip()
    #         if menu_name in final_order_list:
    #             now_cart.add_item(menu_name,menu_price,image_url)
    conn = sqlite3.connect("../data/Comfile_Coffee_DB.db")
    cursor = conn.cursor()
    for order in order_list:
        menu_name=order.strip()
        menu_table = cursor.execute(f"SELECT * FROM menus WHERE menu_name LIKE '%{menu_name}%'").fetchone()
        now_cart.add_item(menu_name,menu_table[-2],menu_table[-1])
    cursor.close()
    return now_cart.items

confirm_order = Tool(
    name="confirm_order",
    description="Moves only the confirmed items from the 'final_order_list' to the cart. The input should be a list of confirmed item names.",
    func=add_Cart
)


tools= [menu_db_sql_tool, chat_retriever_tool,confirm_order]
agent = create_tool_calling_agent(llm=llm,tools=tools,prompt=prompt)

In [414]:
from langchain.agents import AgentExecutor

agent_exe=AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,
    handle_parsing_errors=True,
    max_iterations=5,
    must_use_tools= [menu_db_sql_tool]
)
# result = agent_exe.invoke({'input':"따뜻한 바닐라 라뗴 주세요"})
# print("===Result===")
# print(result['output'])

In [415]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

# session_id 를 저장할 딕셔너리 생성
store = {}


# session_id 를 기반으로 세션 기록을 가져오는 함수
def get_session_history(session_ids):
    if session_ids not in store:  # session_id 가 store에 없는 경우
        # 새로운 ChatMessageHistory 객체를 생성하여 store에 저장
        store[session_ids] = ChatMessageHistory()
    return store[session_ids]  # 해당 세션 ID에 대한 세션 기록 반환


# 채팅 메시지 기록이 추가된 에이전트를 생성합니다.
agent_with_chat_history = RunnableWithMessageHistory(
    agent_exe,
    # 대화 session_id
    get_session_history,
    # 프롬프트의 질문이 입력되는 key: "input"
    input_messages_key="input",
    # 프롬프트의 메시지가 입력되는 key: "chat_history"
    history_messages_key="chat_history",
)

In [ ]:
user_input =""

response = agent_with_chat_history.invoke(
    {"input": user_input},
    # session_id 설정
    config={"configurable": {"session_id": session_id}},
)

response['output']



> Entering new AgentExecutor chain...
주문이 초기화되었습니다. 다시 메뉴를 선택하시겠습니까? 어떤 음료를 원하시는지 말씀해 주세요.

> Finished chain.


'주문이 초기화되었습니다. 다시 메뉴를 선택하시겠습니까? 어떤 음료를 원하시는지 말씀해 주세요.'

In [429]:
now_cart.items

[]

In [428]:
now_cart.clear_cart()

In [171]:
cursor=conn.cursor()
cursor.execute("SELECT * FROM menus WHERE menu_name LIKE '%아메리카노%'").fetchone()[-1]

'data/menu_images/menu_image20.jpg'

In [526]:
print(float(query_buffer[0].split(",")[-2]))
print(query_buffer[0].split(",")[-1])
print(re.sub(r"[^a-zA-Z0-9가-힣/._ ]","",query_buffer[0].split(",")[-1]).strip())
print(re.sub(r"""['"]""","",query_buffer[0].split(",")[1]).strip())
re.sub(r"""['"]""","",query_buffer[0].split(",")[1]).strip()

1800.0
 'data/menu_images/menu_image22.jpg')]
data/menu_images/menu_image22.jpg
ICE 아메리카노


'ICE 아메리카노'

In [72]:
query_buffer

["[(22, 'ICE 아메리카노', '커피 · 더치', '⚬ 용량 : 591ml (20oz) ⚬ 카페인 - 2shot : 156mg/45ml\\r⋆ 고카페인 함유 ⚬ 열량(kcal) : 15 ⚬ 나트륨(mg) : 7.5 ⚬ 탄수화물(g) : 3', 1800.0, 'data/menu_images/menu_image22.jpg')]",
 "[(22, 'ICE 아메리카노', '커피 · 더치', '⚬ 용량 : 591ml (20oz) ⚬ 카페인 - 2shot : 156mg/45ml\\r⋆ 고카페인 함유 ⚬ 열량(kcal) : 15 ⚬ 나트륨(mg) : 7.5 ⚬ 탄수화물(g) : 3', 1800.0, 'data/menu_images/menu_image22.jpg')]",
 "[(6, '딸기가 부라운 붕어', '컴포즈 콤보', '⚬ 구성 : 딸기라떼 + 브라운 붕어 딕만스 ⚬ 용량 : 음료(591ml)+토핑(65g) ⚬ 열량(kcal) : 725.00 ⚬ 나트륨(mg) : 178.26 ⚬ 탄수화물(g) : 107.02 ⚬ 당류(g) : 78.59 ⚬ 포화지방(g) : 20.22 ⚬ 지방(g) : 27.60 ⚬ 단백질(g) : 9.18', 6800.0, 'data/menu_images/menu_image6.jpg'), (9, '붕어는 T입니다', '컴포즈 콤보', '⚬ 구성 : 아이스 밀크티  + 팥 미니 붕어빵 ⚬ 용량 : 음료(591ml) + 토핑(100g) ⚬ 카페인(mg) : 170 ⚬ 열량(kcal) : 591.00 ⚬ 나트륨(mg) : 425.10 ⚬ 탄수화물(g) : 82.40 ⚬ 당류(g) : 56.60 ⚬ 포화지방(g) : 8.90 ⚬ 지방(g) : 16.60 ⚬ 단백질(g) : 12.60', 5800.0, 'data/menu_images/menu_image9.jpg'), (18, '붕어딕만스쿠키', '시즌한정', '⚬ 열량(kcal) : 410 ⚬ 나트륨(mg) : 30.00 ⚬ 탄수화물(g) : 37.33 ⚬ 당류(g) : 22.00 ⚬ 포화지방(g)

In [ ]:
store  

{'2': InMemoryChatMessageHistory(messages=[HumanMessage(content='마시던 거로 주세요', additional_kwargs={}, response_metadata={}), AIMessage(content='마시던 음료에 대해 좀 더 구체적으로 말씀해 주실 수 있나요? 어떤 음료를 드셨는지 알려주시면, 주문을 도와드리겠습니다.', additional_kwargs={}, response_metadata={})])}

In [76]:
query_buffer[2].split(",")

for menu_table in query_buffer:
    print(menu_table)
    for i in range(len(menu_table.split(","))//6):
        menu_price =  float(menu_table.split(",")[i*6+4])
        menu_name = re.sub(r"""['"]""","",menu_table.split(",")[i*6+1]).strip()
        image_url = re.sub(r"[^a-zA-Z0-9가-힣/._ ]","",menu_table.split(",")[i*6+5]).strip()
        print(menu_price,menu_name,image_url)

[(22, 'ICE 아메리카노', '커피 · 더치', '⚬ 용량 : 591ml (20oz) ⚬ 카페인 - 2shot : 156mg/45ml\r⋆ 고카페인 함유 ⚬ 열량(kcal) : 15 ⚬ 나트륨(mg) : 7.5 ⚬ 탄수화물(g) : 3', 1800.0, 'data/menu_images/menu_image22.jpg')]
1800.0 ICE 아메리카노 data/menu_images/menu_image22.jpg
[(22, 'ICE 아메리카노', '커피 · 더치', '⚬ 용량 : 591ml (20oz) ⚬ 카페인 - 2shot : 156mg/45ml\r⋆ 고카페인 함유 ⚬ 열량(kcal) : 15 ⚬ 나트륨(mg) : 7.5 ⚬ 탄수화물(g) : 3', 1800.0, 'data/menu_images/menu_image22.jpg')]
1800.0 ICE 아메리카노 data/menu_images/menu_image22.jpg
[(6, '딸기가 부라운 붕어', '컴포즈 콤보', '⚬ 구성 : 딸기라떼 + 브라운 붕어 딕만스 ⚬ 용량 : 음료(591ml)+토핑(65g) ⚬ 열량(kcal) : 725.00 ⚬ 나트륨(mg) : 178.26 ⚬ 탄수화물(g) : 107.02 ⚬ 당류(g) : 78.59 ⚬ 포화지방(g) : 20.22 ⚬ 지방(g) : 27.60 ⚬ 단백질(g) : 9.18', 6800.0, 'data/menu_images/menu_image6.jpg'), (9, '붕어는 T입니다', '컴포즈 콤보', '⚬ 구성 : 아이스 밀크티  + 팥 미니 붕어빵 ⚬ 용량 : 음료(591ml) + 토핑(100g) ⚬ 카페인(mg) : 170 ⚬ 열량(kcal) : 591.00 ⚬ 나트륨(mg) : 425.10 ⚬ 탄수화물(g) : 82.40 ⚬ 당류(g) : 56.60 ⚬ 포화지방(g) : 8.90 ⚬ 지방(g) : 16.60 ⚬ 단백질(g) : 12.60', 5800.0, 'data/menu_images/menu_image9.jpg'), (18, '붕어딕만스쿠키

In [ ]:
# 벡터 DB에 세션 데이터 저장
def save_session_to_vector_db(session_id):
    if session_id in store:
        chat_history = store[session_id].messages  # 현재 세션의 대화 내용 가져오기
        for msg in chat_history:
            vector_store.add_texts([msg.content], metadatas=[{"session_id": session_id, "role": msg.type}])
        print(f"✅ Session {session_id} data saved to vector DB.")

# 현재 활성화된 세션 초기화
def clear_session(session_id):
    if session_id in store:
        del store[session_id]  # 세션 데이터 삭제
        print(f"✅ Session {session_id} cleared.")

In [ ]:
save_session_to_vector_db(session_id)
clear_session(session_id)

✅ Session 2 data saved to vector DB.
✅ Session 2 cleared.


In [94]:
vector_store.get(where={"session_id": session_id})

{'ids': ['59a01ff1-6aa7-41a3-9078-c9011a0aa1f5',
  'c6d10195-ae9f-4f08-a3e6-49ec4d08920b',
  '3ffc3fb5-efb9-473f-90c5-73d8af9d64da',
  '83faf17c-50d6-4265-80af-c069da450b3d',
  '2948f651-2be0-41b8-9ae8-fe1334de3dc6',
  '6bc9086b-7cf8-43fb-b534-7bb1c44c408b',
  'c9cca5cd-153f-4161-9498-3434d0f22588',
  'a9fce903-632c-4ea0-9dfe-a712804598cd',
  '1d6eac8b-f790-447b-ba83-3c297df63892',
  '1d5423d8-8403-476e-8a9d-b270251b0711',
  '0c35569a-2fee-4eb2-a347-7d85031d9289',
  '6ea8ab46-f9e0-492a-a897-4049c172218e',
  '65dc61b5-4275-4d59-afc0-2ecab5ec0d8a',
  '38c6930c-ca29-4396-bbac-0c241257b566'],
 'embeddings': None,
 'documents': ['카페라떼 가격이 얼마인가요?',
  '카페라떼는 2,900원입니다.',
  '카페라떼 가격이 얼마인가요?',
  '카페라떼는 2,900원입니다.',
  '카페라떼 가격이 얼마인가요?',
  '카페라떼는 2,900원입니다.',
  '마시던 거로 주세요',
  '이전에 주문하신 음료는 카페라떼이며, 가격은 2,900원입니다. 이 음료로 주문하시겠습니까?',
  '카페라떼 가격이 얼마인가요?',
  '카페라떼는 2,900원입니다.',
  '마시던 거로 주세요',
  '마시던 음료에 대해 좀 더 구체적으로 말씀해 주실 수 있나요? 어떤 음료를 드셨는지 알려주시면, 주문을 도와드리겠습니다.',
  '카페라떼 가격이 얼마인가요?',
  '카페라떼는 2,900원

In [378]:
conn = sqlite3.connect("../data/Comfile_Coffee_DB.db")
cursor = conn.cursor()
result = cursor.execute("SELECT * FROM menus WHERE menu_name LIKE '%아아%'").fetchone()
print([result[1],result[4],result[5]])
conn.close()

['아아~ 초코볼~', 3700.0, 'data/menu_images/menu_image7.jpg']
